<a href="https://colab.research.google.com/github/sridivya1/Covid19-Prediction/blob/main/CNN_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil

In [ ]:
base = '/content/drive/My Drive/Covid_DS'


In [ ]:
directories = [d for d in os.listdir(base) if os.path.isdir(os.path.join(base, d))]

# Print the list of directories
print("Directories in", base, ":")
for directory in directories:
    print(directory)

In [ ]:
base = '/content/drive/My Drive/Covid_DS/DATASET'

In [ ]:
#1.Loading the data

In [ ]:
# Define the class labels
class_labels = ["covid", "pneumonia", "normal"]

In [ ]:
data = []

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def load_and_preprocess_data_in_batches(base_dir, class_labels, batch_size=100):
    batch_data = []

    for label in class_labels:
        class_path = os.path.join(base_dir, label)

        # Loop through the images in the class folder
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
            img = cv2.resize(img, (100, 100)) # Resize to a smaller size (adjust as needed)
            img = img / 255.0 # Normalize

            # Append image and its corresponding label index to the batch data
            batch_data.append([img, class_labels.index(label)])

            # Check if batch size is reached, then add to main data and clear batch data
            if len(batch_data) == batch_size:
                data.extend(batch_data)
                batch_data = []

    # Add remaining data if any
    if batch_data:
        data.extend(batch_data)


In [ ]:
load_and_preprocess_data_in_batches(base, class_labels)
X = np.array([item[0] for item in data])
y = np.array([item[1] for item in data])

In [ ]:
import matplotlib.pyplot as plt
import random

# Assuming you have already loaded and preprocessed the data in batches
# data = [ [img1, label1], [img2, label2], ... ]

# Shuffle the data
random.shuffle(data)

# Visualize some random sample images from all classes
num_samples = 15
num_rows = (num_samples + 3) // 4  # Calculate the number of rows needed

fig, axes = plt.subplots(num_rows, 4, figsize=(20, 5*num_rows))  # Increase figsize

for i in range(num_samples):
    row = i // 4
    col = i % 4
    img, label = data[i]
    axes[row, col].imshow(img)
    axes[row, col].set_title(class_labels[label])
    axes[row, col].axis('off')

# Hide empty subplots
for i in range(num_samples, num_rows*4):
    row = i // 4
    col = i % 4
    axes[row, col].axis('off')

plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))


In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

In [ ]:
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_labels), activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 10  # Number of epochs
batch_size = 32  # Batch size

history=model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")


In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.save('CNN_model.h5')